This works

In [26]:
import graphix.command
from graphix.random_objects import Circuit, rand_circuit
from graphix.states import BasicStates
from graphix.sim.density_matrix import DensityMatrixBackend
from graphix.sim.statevec import StatevectorBackend, Statevec
from graphix.pauli import Pauli
from graphix.fundamentals import IXYZ
import stim
from graphix.noise_models import DepolarisingNoiseModel

from veriphix.client import Client, Secrets, CircuitUtils
import veriphix.client
import numpy as np
import random

In [36]:
nqubits = 4
depth = 80
circuit = rand_circuit(nqubits, depth)
pattern = circuit.transpile().pattern 
pattern.minimize_space()

print(f"Number of nodes in the pattern : {pattern.n_node}")

## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))


Number of nodes in the pattern : 1074


In [37]:
secrets = Secrets(r=True, a=True, theta=True)
client = Client(pattern=pattern, secrets=secrets)
runs = client.create_test_runs()

backend = DensityMatrixBackend()


# Noiseless computation run
# client.delegate_pattern(backend=backend)

# Noiseless trap run
noiseless_model = DepolarisingNoiseModel(entanglement_error_prob=0)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(backend=backend, run=rd_run, noise_model=noiseless_model)
print(f"Trap outcome of noiseless trap run: {sum(trap_outcomes)}") # should output 0 (noiseless simulation)

# (ultra) Noisy trap run
ultra_noisy_model = DepolarisingNoiseModel(entanglement_error_prob=1)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(backend=backend, run=rd_run, noise_model=ultra_noisy_model)
print(f"Trap outcome of noisy trap run: {sum(trap_outcomes)}") # should NOT output 0 (noisy simulation)


Trap outcome of noiseless trap run: 0
Trap outcome of noisy trap run: 234
